# **환경 설정**

In [ ]:
%%capture --no-stderr
!pip install langchain_community tiktoken langchain-openai chromadb langchain langgraph

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# **문서 정의**

In [ ]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# 크롤링할 URL 지정
url = "https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel"

# 해당 페이지를 재귀적으로 크롤링
loader = RecursiveUrlLoader(
    url=url,
    max_depth=20,
    extractor=lambda x: Soup(x, "html.parser").text
)

# 지정된 URL에서 크롤링한 문서를 'docs' 변수에 저장
docs = loader.load()

# 크롤링된 문서를 'source' 메타데이터를 기준으로 정렬
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))

# 모든 문서의 내용을 하나의 문자열로 연결
concatenated_content = "\n\n\n --- \n\n\n".join([doc.page_content for doc in d_reversed])

# **코드 생성**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# LLM이 LCEL 전문가로서 사용자의 질문에 답변하도록 지시하는 프롬프트 정의
system = """
당신은 LCEL(LangChain expression language) 전문가인 코딩 어시스턴트입니다.
다음은 필요한 LCEL 문서 전문입니다:
--------
{context}
--------
위에 제공된 문서를 기반으로 사용자 질문에 답변하세요.
제공하는 코드는 실행 가능해야 하며, 필요한 모든 import 문과 변수들이 정의되어 있어야 합니다.
답변을 다음과 같은 구조로 작성하세요:
1. prefix : 문제와 접근 방식에 대한 설명
2. imports : 코드 블록 import 문
3. code : import 문을 제외한 코드 블록
4. description : 질문에 대한 코드 스키마

다음은 사용자 질문입니다:
"""

# 시스템 메시지와 사용자의 질문을 포함한 템플릿 작성
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("placeholder", "{messages}"),
    ]
)

# 코드 출력을 구조화하기 위한 데이터 모델 정의
class code(BaseModel):
  prefix: str = Field(description="문제와 접근 방식에 대한 설명")
  imports: str = Field(description="코드 블록 import 문")
  code: str = Field(description="import 문을 제외한 코드 블록")
  description: str = Field(description="질문에 대한 코드 스키마")

# 코드 생성을 위한 LLM 정의
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# 프롬프트, 구조화된 LLM 출력을 결합하여 RAG 체인 생성
code_gen_chain = code_gen_prompt | llm.with_structured_output(code)

In [ ]:
question = "LCEL로 RAG 체인을 어떻게 만들어?"
solution = code_gen_chain.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)

print(solution)

# **상태**

In [ ]:
from typing import List, TypedDict

class GraphState(TypedDict):
  error: str
  messages: List
  generation: str
  iterations: int